This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [6]:
import numpy as np 
import pandas as pd

In [15]:
DataFrame=pd.read_csv('data/data_000637.txt',delimiter=',')[0:30000]
DataFrame

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
29995,1,1,60,3869200466,356,11
29996,1,0,64,3869200466,347,4
29997,1,0,139,3869200466,357,0
29998,1,0,61,3869200466,353,10


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [16]:
x = DataFrame["BX_COUNTER"].max()+1
print("The number of BX in a ORBIT is : ",x)

The number of BX in a ORBIT is :  3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [23]:
data_taking_last = DataFrame.loc[len(DataFrame)-1,'ABS_TIMES(ns)'] - DataFrame.loc[0,'ABS_TIMES(ns)']
print('data takeing lasted:', data_taking_last, ' ns')




data takeing lasted: 1015193.5  ns


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [24]:
DataFrame['ABS_TIMES(ns)']=DataFrame['TDC_MEAS']*25/30+DataFrame['BX_COUNTER']*25 + DataFrame['ORBIT_CNT']*x
print(DataFrame)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
29995     1     1           60  3869200466         356        11   
29996     1     0           64  3869200466         347         4   
29997     1     0          139  3869200466         357         0   
29998     1     0           61  3869200466         353        10   
29999     1     0          119  3869200466         357         5   

       ABS_TIMES(ns)  
0       1.378983e+13  
1       1.378983e+13  
2       1.378983e+13  
3       1.378983e+13  
4       1.378983e+13  
...              ...  
29995   1.378983e+13  

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel